In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
def relative_response_time_plot(framework_concurrency):
    serverless_results = pd.read_csv("../serverless_results.csv")
    framework_results = pd.read_csv("../results.csv")
    framework_results = framework_results[framework_results["concurrency"] == framework_concurrency]
    framework_results = framework_results[framework_results["framework"] != "express_big_instance_with_cluster"]
    framework_results = framework_results[framework_results["framework"] != "express_big_instance_without_cluster"]
    framework_results = framework_results[framework_results["framework"] != "express_without_cluster"]
    framework_results = framework_results[framework_results["framework"] != "spring_large_instance"]
    results = pd.concat([serverless_results, framework_results])
    results = results.sort_values(by=["concurrency", "framework"])
    results = results.drop(columns=["concurrency"])
    
    avg_response_time_spring = results[results["framework"] == "spring"].groupby("endpoint")["avg_response_time"].max()
    avg_response_time_express = results[results["framework"] == "express_with_cluster"].groupby("endpoint")["avg_response_time"].max()
    avg_response_time_django = results[results["framework"] == "django"].groupby("endpoint")["avg_response_time"].max()
    avg_response_time_serverless = results[results["framework"] == "serverless"].groupby("endpoint")["avg_response_time"].max()
    
    avg_response_time_spring = avg_response_time_spring.rename(index={"get_price": "getPrice"})
    avg_response_time_express = avg_response_time_express.rename(index={"get_price": "getPrice"})
    avg_response_time_django = avg_response_time_django.rename(index={"get_price": "getPrice"})
    avg_response_time_serverless = avg_response_time_serverless.rename(index={"get_price": "getPrice"})
    
    avg_response_time_spring = avg_response_time_spring.reindex(["echo", "getPrice", "compute", "parse", "query"])
    avg_response_time_express = avg_response_time_express.reindex(["echo", "getPrice", "compute", "parse", "query"])
    avg_response_time_django = avg_response_time_django.reindex(["echo", "getPrice", "compute", "parse", "query"])
    avg_response_time_serverless = avg_response_time_serverless.reindex(["echo", "getPrice", "compute", "parse", "query"])
    
    avg_response_time_express = avg_response_time_express / avg_response_time_django
    avg_response_time_spring = avg_response_time_spring / avg_response_time_django
    avg_response_time_serverless = avg_response_time_serverless / avg_response_time_django
    avg_response_time_django = avg_response_time_django / avg_response_time_django

    avg_response_time_express["average"] = avg_response_time_express.mean()
    avg_response_time_spring["average"] = avg_response_time_spring.mean()
    avg_response_time_serverless["average"] = avg_response_time_serverless.mean()
    avg_response_time_django["average"] = avg_response_time_django.mean()
    
    print("Average relative response time for Express: ", avg_response_time_express["average"])
    print("Average relative response time for Spring: ", avg_response_time_spring["average"])
    print("Average relative response time for Serverless: ", avg_response_time_serverless["average"])
    print("Average relative response time for Django: ", avg_response_time_django["average"])

    fig = plt.figure(figsize=(6, 3), dpi=200)
    barWidth = 0.2
    r1 = np.arange(len(avg_response_time_spring))
    r2 = [x + barWidth for x in r1]
    r3 = [x + barWidth for x in r2]
    r4 = [x + barWidth for x in r3]
    plt.bar(r1, avg_response_time_django, width=barWidth, label="Django")
    plt.bar(r2, avg_response_time_express, width=barWidth, label="Express")
    plt.bar(r3, avg_response_time_spring, width=barWidth, label="Spring")
    plt.bar(r4, avg_response_time_serverless, width=barWidth, label="Serverless")
    
    handles, labels = plt.gca().get_legend_handles_labels()
    order = [0, 1, 2, 3]
    plt.legend([handles[idx] for idx in order], [labels[idx] for idx in order], ncol=4)
    
    plt.xticks([r + barWidth for r in range(len(avg_response_time_spring))], avg_response_time_spring.index)
    plt.xlabel("Endpoint", fontsize=11)
    plt.ylabel("Relative mean response time", fontsize=11)
    plt.ylim(top=1.2)

    fig_dir = "../graphs/serverless_vs_frameworks/"
    os.makedirs(fig_dir, exist_ok=True)
    plt.savefig(fig_dir + f"/relative_response_times_framework_concurrency_{framework_concurrency}.png",dpi=fig.dpi, bbox_inches = 'tight')
    plt.show()
    plt.close()

In [ ]:
relative_response_time_plot(1)

In [ ]:
relative_response_time_plot(64)